In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [2]:
import keras
import keras.ops as K
from keras.layers import Input, Flatten, Dense
from keras.optimizers import Adam
from keras.metrics import BinaryAccuracy

# from keras.models import Sequential
from deel.lip.model import Sequential

from deel.lip.layers import (
    SpectralDense,
    SpectralConv2D,
    ScaledL2NormPooling2D,
    FrobeniusDense,
)
from deel.lip.activations import GroupSort, GroupSort2
from deel.lip.losses import HKR, KR, HingeMargin, MulticlassHKR, MulticlassKR

import numpy as np
import decomon

from data_processing import load_data, select_data_for_radius_evaluation_MNIST08
from radius_evaluation_tools import compute_binary_certificate, starting_point_dichotomy

In [3]:
x_train, x_test, y_train, y_test, y_test_ord = load_data("MNIST08")

In [4]:
model_path = "/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output.keras"
model = keras.models.load_model(model_path)
model.compile(
   
    loss=HKR(
        alpha=10.0, min_margin=1.0
    ),  # HKR stands for the hinge regularized KR loss
    metrics=[
        # KR,  # shows the KR term of the loss
        HingeMargin(min_margin=1.0),  # shows the hinge term of the loss
    ],
    optimizer=Adam(learning_rate=0.001),)

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 12 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
model_bis_path = "/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output_converted_4logits.keras"
model_bis = keras.models.load_model(model_bis_path)
model_bis.compile(
        # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
        # note also in the case of lipschitz networks, more robustness require more parameters.
        loss=MulticlassHKR(alpha=100, min_margin=0.25),
        optimizer=Adam(1e-4),
        metrics=["accuracy", MulticlassKR()],)

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [6]:
from decomon.layers import DecomonLayer
from decomon.models import clone
from lipschitz_custom_tools import *
from decomon.perturbation_domain import BallDomain
from decomon import get_lower_noise, get_range_noise, get_upper_noise

In [7]:
images, labels, idx_list = select_data_for_radius_evaluation_MNIST08(x_test, y_test_ord, model_bis)

torch.Size([100, 1, 28, 28])
torch.Size([100, 1, 28, 28])


In [8]:
from keras.models import Model

In [9]:
import pdb

In [10]:
class DecomonGroupSort2(DecomonLayer):
    layer : GroupSort2
    increasing = True
    def get_affine_bounds(self, lower, upper):
        (W_low, b_low), (W_up, b_up) = affine_bound_groupsort_output_keras(lower, upper)
        W_low = K.transpose(W_low,(0,2,1))
        W_up = K.transpose(W_up,(0,2,1))
        return W_low, b_low, W_up, b_up

In [ ]:
# class DecomonGroupSort2(DecomonLayer):
#     layer : GroupSort2
#     increasing = True
#     def __init__(self,**kwargs):
#         super().__init__(**kwargs)
#         self.model = Model(model.inputs, self.layer.input)
#         self.lip = 1
        
#     def get_affine_bounds(self, lower, upper):
#         (W_low, b_low), (W_up, b_up) = affine_bound_groupsort_output_keras(lower, upper)
#         W_low = K.transpose(W_low,(0,2,1))
#         W_up = K.transpose(W_up,(0,2,1))
#         return W_low, b_low, W_up, b_up
    
#     def call(self, inputs, training=False):
        
#         (
#             affine_bounds_to_propagate,
#             constant_oracle_bounds,
#             perturbation_domain_inputs,
#         ) = self.inputs_outputs_spec.split_inputs(inputs=inputs)

#         x = perturbation_domain_inputs
#         eps = self.perturbation_domain.eps
#         pdb.set_trace()
#         lower_lip = self.model(x) - self.lip*eps
#         upper_lip = self.model(x) + self.lip*eps

#         lipschitz_bounds = [lower_lip, upper_lip]
#         affine_bounds_propagated = self.call_backward(
#             affine_bounds_to_propagate=affine_bounds_to_propagate, constant_oracle_bounds=lipschitz_bounds
#         )
#         return self.inputs_outputs_spec.flatten_outputs(affine_bounds_propagated)
        

#         #since model is 1-Lipschitz we compute tighter upper and lower bounds

In [11]:
lip_certificate = compute_binary_certificate(images, model)

In [12]:
lip_certificate[:10]

tensor([1.6613, 3.0277, 3.0674, 4.6951, 4.3250, 3.6832, 3.1318, 3.9830, 4.2999,
        2.5395], device='cuda:0', grad_fn=<SliceBackward0>)

In [13]:
def compute_Decomon_certificates(i, epsilon, inputs, model):
    perturbation_domain = BallDomain(eps=epsilon, p=2)
    decomon_model = clone(model, mapping_keras2decomon_classes={GroupSort2:DecomonGroupSort2}, final_ibp=True, final_affine=False, perturbation_domain=perturbation_domain, method='crown')
    # upper_test_ = get_upper_noise(decomon_model,  inputs[i:i+1], eps=epsilon, p=2)[:, 0]
    # lower_test_ = get_lower_noise(decomon_model, inputs[i:i+1], eps=epsilon, p=2)[:, 0]
    # return lower_test_, upper_test_
    return decomon_model.predict(inputs[i:i+1])

In [14]:
perturbation_domain = BallDomain(eps=1, p=2)
decomon_model = clone(model, mapping_keras2decomon_classes={GroupSort2:DecomonGroupSort2}, final_ibp=True, final_affine=False, perturbation_domain=perturbation_domain, method="crown")
lower, upper = decomon_model.predict(images[0:1])

2025-05-06 15:29:17.153251: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746538157.173543   18673 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746538157.179893   18673 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-06 15:29:17.199728: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step


/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['perturbation_domain_input_lipModel']
Received: inputs=Tensor(shape=torch.Size([1, 1, 28, 28]))
  warnings.warn(msg)


In [15]:
model.predict(images[0:1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


array([[-1.6612895]], dtype=float32)

In [16]:
print(compute_Decomon_certificates(0, 1, images, model))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
[array([[-3.5799596]], dtype=float32), array([[1.1188315]], dtype=float32)]


In [17]:
model.predict(images[8:9])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[-4.299889]], dtype=float32)

In [33]:
def single_compute_decomon_radius(idx, images, targets, model, n_iter = 10):
    image = images[idx:idx+1]
    target = targets[idx:idx+1]
    print("class of the input : ", target[0])
    # certificate = certificates[idx:idx+1]
    # We use dichotomy algorithm to fine the smallest optimistic radius
    # We start from the closest point with different class
    d_up = starting_point_dichotomy(idx, images, targets)
    eps_working = d_low = 0
    for _ in range(n_iter):
        eps_current = (d_up+d_low)/2
        print(eps_current)
        perturbation_domain = BallDomain(eps=eps_current, p=2)
        decomon_model = clone(model, mapping_keras2decomon_classes={GroupSort2:DecomonGroupSort2}, final_ibp=True, final_affine=False, perturbation_domain=perturbation_domain, method="crown")
        lower, upper = decomon_model.predict(image)

        # lower, upper = compute_Decomon_certificates(idx, eps_current, images, model)
        if (target==0 and upper<=0) or (target==1 and lower>=0):
            print("working", upper, lower)
            eps_working = d_low = eps_current
        else:
            print("not working", upper, lower)
            d_up = eps_current
    return eps_working

In [34]:
idx = 199
eps = single_compute_decomon_radius(idx, images, labels, model)
print(lip_certificate[idx], eps)

class of the input :  tensor(1, device='cuda:0')
tensor(5.0054, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
not working [[42.61066]] [[-23.46467]]
tensor(2.5027, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['perturbation_domain_input_lipModel']
Received: inputs=Tensor(shape=torch.Size([1, 1, 28, 28]))
  warnings.warn(msg)


not working [[17.259666]] [[-7.0412064]]
tensor(1.2513, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
working [[7.6462293]] [[1.4089662]]
tensor(1.8770, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
not working [[11.850521]] [[-2.9264975]]
tensor(1.5642, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
not working [[9.273465]] [[-0.45567703]]
tensor(1.4078, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
working [[8.378182]] [[0.58449364]]
tensor(1.4860, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
working [[8.82137]] [[0.0692476]]
tensor(1.5251, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
not working [[9.048304]] [[-0.19288075]]
tensor(1.5055, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
not working [[8.934013]] [[-0.06105053]]
tensor(1.4957, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
working [[8.8764]] [[0.00535846]]
tensor(4.4950, device='cuda:0', grad_fn=<SelectBackward0>) tensor(1.4957, device='cuda:0')


In [31]:
compute_Decomon_certificates(199, 1.5, images, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


[array([[-0.0236156]], dtype=float32), array([[8.901539]], dtype=float32)]

In [32]:
idx = 0
eps = single_compute_decomon_radius(idx, images, labels, model)
print(lip_certificate[idx], eps)

class of the input :  tensor(0, device='cuda:0')
tensor(4.6030, device='cuda:0')


TypeError: TorchTrainer.predict() got an unexpected keyword argument 'eps'

In [25]:
list_eps = []
list_certif = []
for idx in range(100,110):
    list_eps.append(single_compute_decomon_radius(idx, images, labels, model))
    list_certif.append(lip_certificate[idx])

class of the input :  tensor(1, device='cuda:0')
tensor(4.2620, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


not working [[35.14774]] [[-19.776806]]
tensor(2.1310, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
not working [[13.840949]] [[-5.616832]]
tensor(1.0655, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
working [[5.861185]] [[1.2494501]]
tensor(1.5982, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
not working [[9.149612]] [[-1.9857818]]
tensor(1.3319, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
not working [[7.3681545]] [[-0.09885299]]
tensor(1.1987, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
working [[6.5584226]] [[0.6675036]]
tensor(1.2653, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
working [[6.940564]] [[0.32510614]]
tensor(1.2986, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
working [[7.1377616]] [[0.14247811]]
tensor(1.3152, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
working [[7.250029]] [[0.02541304]]
tensor(1.3235, device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
not working [[7.307299]] [[-

In [26]:
list_eps

[tensor(1.3152, device='cuda:0'),
 tensor(1.0265, device='cuda:0'),
 tensor(1.0275, device='cuda:0'),
 tensor(1.5191, device='cuda:0'),
 tensor(1.1402, device='cuda:0'),
 tensor(0.5239, device='cuda:0'),
 tensor(1.0790, device='cuda:0'),
 tensor(1.2535, device='cuda:0'),
 tensor(0.9385, device='cuda:0'),
 tensor(0.7787, device='cuda:0')]

In [27]:
list_certif

[tensor(3.5752, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(3.2317, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(2.7815, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(4.9184, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(2.7873, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(1.1270, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(3.9001, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(4.5482, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(2.9653, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(2.2265, device='cuda:0', grad_fn=<SelectBackward0>)]

# CE qui suit est inutile

In [ ]:
box = np.zeros((1,2,1,28,28))

In [ ]:
box[:,1] += 0.1

In [ ]:
decomon_model.predict(box)

NameError: name 'decomon_model' is not defined

In [ ]:
class Sqrt(keras.layers.Layer):
    def call(self, inputs):
        return K.sqrt(inputs)

class Square(keras.layers.Layer):
    def call(self, inputs):
        return K.square(inputs)

In [ ]:
class DecomonSqrt(DecomonLayer):
    layer : Sqrt
    increasing = True
    def get_affine_bounds(self, lower, upper):
        (W_low, b_low), (W_up, b_up) = affine_bound_sqrt_output_keras(lower, upper)
        return W_low, b_low, W_up, b_up
class DecomonSquare(DecomonLayer):
    layer : Square
    def get_affine_bounds(self, lower, upper):
        (W_low, b_low), (W_up, b_up) = affine_bound_square_output_keras(lower, upper)
        return W_low, b_low, W_up, b_up
    def forward_ibp_propagate(self, lower, upper):
        return self.layer(lower), self.layer(upper)

In [ ]:
sqrt = Sqrt()

In [ ]:
sqrt.predict(4)

# Test GS2 decomon

## Cas 1 : z0 > 0

In [ ]:
from deel.lip.model import Sequential

In [ ]:
modelgs = Sequential([GroupSort2()])

In [ ]:
input = K.expand_dims(K.array([2, -2], dtype='float32'), axis=0)

In [ ]:
input.shape

torch.Size([1, 2])

In [ ]:
low = K.expand_dims(K.array([1, -3], dtype='float32'), axis=0)
up = K.expand_dims(K.array([3, -1], dtype='float32'), axis=0)
bounds =  K.expand_dims(K.concatenate((low,up), axis=0), axis=0)

In [ ]:
modelgs(K.ones_like(input))

tensor([[1., 1.]], device='cuda:0')

In [ ]:
bounds.shape

torch.Size([1, 2, 2])

In [ ]:
decomon_model = clone(modelgs, mapping_keras2decomon_classes={GroupSort2:DecomonGroupSort2}, final_ibp=False, final_affine=True, method='crown')

2025-04-30 10:06:08.898731: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746000368.918191   27029 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746000368.924250   27029 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-30 10:06:08.944125: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
W_low, b_low, W_up, b_up = decomon_model.predict(bounds)

tensor([[[[0.8182],
          [0.1818]]]], device='cuda:0')
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['perturbation_domain_input_sequential']
Received: inputs=Tensor(shape=torch.Size([1, 2, 2]))
  warnings.warn(msg)


In [ ]:
print(W_low, '\n',W_up, '\n',b_low, '\n',b_up)

[[[0. 1.]
  [1. 0.]]] 
 [[[0. 1.]
  [1. 0.]]] 
 [[0. 0.]] 
 [[ 0. -0.]]


In [ ]:
modelgs(input)

tensor([[-2.,  2.]], device='cuda:0')

In [ ]:
W_low[0,:] @ K.convert_to_numpy(input)[0,:] + b_low

array([[-2.,  2.]], dtype=float32)

In [ ]:
W_up[0,:] @ K.convert_to_numpy(input)[0,:] + b_up

array([[-2.,  2.]], dtype=float32)

## Cas 2 z0 <= 0 <= z1

In [ ]:
input = K.expand_dims(K.array([-3, 5], dtype='float32'), axis=0)

In [ ]:
low = K.expand_dims(K.array([-4, 1], dtype='float32'), axis=0)
up = K.expand_dims(K.array([3, 5], dtype='float32'), axis=0)
bounds =  K.expand_dims(K.concatenate((low,up), axis=0), axis=0)

In [ ]:
W_low, b_low, W_up, b_up = decomon_model.predict(bounds)

NameError: name 'decomon_model' is not defined

In [ ]:
print(W_low, '\n',W_up, '\n',b_low, '\n',b_up)

[[[0.8181818  0.        ]
  [0.18181819 1.        ]]] 
 [[[1.         0.18181819]
  [0.         0.8181818 ]]] 
 [[-1.6363636  0.       ]] 
 [[0.        1.6363636]]


On voit dans le code suivant qu'il y a un souci :

In [ ]:
modelgs(input)

tensor([[-3.,  5.]], device='cuda:0')

In [ ]:
W_low[0,:] @ K.convert_to_numpy(input)[0,:] + b_low

array([[-4.090909 ,  4.4545455]], dtype=float32)

In [ ]:
W_up[0,:] @ K.convert_to_numpy(input)[0,:] + b_up

array([[-2.090909 ,  5.7272725]], dtype=float32)